In [1]:
#                                                                        
# ost40_exact_test.py                                                    
#                                                                        
                                                                         
#=============                                                           
# 準備                                                                   
#=============                                                           
                                                                         
# cell ==================================                                
                                                                         
import pandas as pd                                                      
import numpy as np                                                       
from scipy.special import comb                                           
import math                                                              
from neal import SimulatedAnnealingSampler                               
from pyqubo import Array, Constraint, Placeholder, solve_qubo            
import itertools                                                         
import random                                                            
import matplotlib.pyplot as plt                                          
import timeit                                                            
                                                                         
#=========================================                               
                                                                         

In [5]:
#=============                              
# 関数定義                                  
#=============                              
                                            
# make_t_list_columns_num_samples           
# は正確検定の表の周辺和を計算しているので、
# 次の関数に置き換え                        
def calc_marginals(df):                     
    return np.array([                       
        sum(df['Y']),                      
        np.dot(df['Y'], df['LI']),         
        np.dot(df['Y'], df['SEX']),        
        np.dot(df['Y'], df['AOP']),        
    ])                                      
                                            

In [6]:
                                                            
#=============                                              
# デバッグ                                                  
#=============                                              
                                                            
# cell ==================================                   
                                                            
# ハミルトニアンを途中まで作成                              
df = pd.read_csv("../../input/ost40.csv", sep=',', index_col=0)       
t_list = calc_marginals(df)                                 
                                                            
y = Array.create('y', shape=df.shape[0], vartype='BINARY')  
H = (sum(y) - t_list[0])**2                                 
                                                            
#========================================                   

In [8]:
#========================================           
                                                    
# ハミルトニアンをQUBOにコンパイルしてサンプリング  
# その際、QUBOのバイアスを表示                      
sa_sampler = SimulatedAnnealingSampler() 
q, b = H.compile().to_qubo()                        
print("offset value: ", b)                          
sa_sampler.sample(q)                           
                                                    
#========================================           
                                                    

offset value:  625.0


SampleSet(rec.array([([0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0], -625., 1)],
          dtype=[('sample', 'i1', (40,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), ['y[0]', 'y[10]', 'y[11]', 'y[12]', 'y[13]', 'y[14]', 'y[15]', 'y[16]', 'y[17]', 'y[18]', 'y[19]', 'y[1]', 'y[20]', 'y[21]', 'y[22]', 'y[23]', 'y[24]', 'y[25]', 'y[26]', 'y[27]', 'y[28]', 'y[29]', 'y[2]', 'y[30]', 'y[31]', 'y[32]', 'y[33]', 'y[34]', 'y[35]', 'y[36]', 'y[37]', 'y[38]', 'y[39]', 'y[3]', 'y[4]', 'y[5]', 'y[6]', 'y[7]', 'y[8]', 'y[9]'], {'beta_range': [0.02829172165550797, 9.210340371976184], 'beta_schedule_type': 'geometric'}, 'BINARY')

In [20]:
#========================================               
                                                        
# ハミルトニアンをBQMにコンパイルしてサンプリング       
bqm = H.compile().to_bqm()                        
sa_sampler.sample(bqm)                                  
                                                        
#========================================               
                                                        

SampleSet(rec.array([([1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1], 0., 1)],
          dtype=[('sample', 'i1', (40,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), ['y[0]', 'y[10]', 'y[11]', 'y[12]', 'y[13]', 'y[14]', 'y[15]', 'y[16]', 'y[17]', 'y[18]', 'y[19]', 'y[1]', 'y[20]', 'y[21]', 'y[22]', 'y[23]', 'y[24]', 'y[25]', 'y[26]', 'y[27]', 'y[28]', 'y[29]', 'y[2]', 'y[30]', 'y[31]', 'y[32]', 'y[33]', 'y[34]', 'y[35]', 'y[36]', 'y[37]', 'y[38]', 'y[39]', 'y[3]', 'y[4]', 'y[5]', 'y[6]', 'y[7]', 'y[8]', 'y[9]'], {'beta_range': [0.02829172165550797, 9.210340371976184], 'beta_schedule_type': 'geometric'}, 'BINARY')

In [28]:
columns = list(df.columns)
#========================================                                    
                                                                             
# ハミルトニアンを全て作成してサンプリング                                   
# 元のデータと周辺和が一致するようにyに制約を加える                          
                                                                             
H = (sum(y) - t_list[0])**2                                                  
t1 = 18                                                                      
H += (np.dot(df[columns[1]], y) - t1)**2 ### ???? ここは何をしている？       
# for j in range(2, len(columns)):                                           
#     H_plus = (np.dot(df[columns[j]], y) - t_list[j])**2                    
#     H += H_plus                                                            
# ここは次のように書いてもよい。                                             
for j in range(2, len(t_list)):                                              
    v = df.to_numpy()[:,j] # 目的の行を取り出す                              
    H += (y.dot(v) - t_list[j]) ** 2                                         
                                                                             
bqm = H.compile().to_bqm()                                             
sa_sampler.sample(bqm)                                                       
                                                                             
#========================================                                                                  

SampleSet(rec.array([([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1], 0., 1)],
          dtype=[('sample', 'i1', (40,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), ['y[0]', 'y[10]', 'y[11]', 'y[12]', 'y[13]', 'y[14]', 'y[15]', 'y[16]', 'y[17]', 'y[18]', 'y[19]', 'y[1]', 'y[20]', 'y[21]', 'y[22]', 'y[23]', 'y[24]', 'y[25]', 'y[26]', 'y[27]', 'y[28]', 'y[29]', 'y[2]', 'y[30]', 'y[31]', 'y[32]', 'y[33]', 'y[34]', 'y[35]', 'y[36]', 'y[37]', 'y[38]', 'y[39]', 'y[3]', 'y[4]', 'y[5]', 'y[6]', 'y[7]', 'y[8]', 'y[9]'], {'beta_range': [0.010663802777845312, 9.210340371976184], 'beta_schedule_type': 'geometric'}, 'BINARY')